In [ ]:
import os
import glob
import numpy as np
from shutil import copyfile
import random
import pandas as pd
import json
import shutil


# Data sampling

#### This function samples data required for Form Recognizer Training.
It sampels such that the file count is < file_limit and Total sample size is < size_limit (in MB)

In [ ]:
def sample_training_data(src_fld, dst_fld, file_limit=500, size_limit=50):

    fls = glob.glob(os.path.join(src_fld,"*.pdf"))

    #sample and copy files
    if len(fls) > file_limit:
        flist = random.sample(fls,file_limit)
        print("sampling files")
    else:
        flist = fls
        print("file count <",file_limit, "considering all files")

    for fl in flist:
        flname = os.path.basename(fl)
        copyfile(os.path.join(src_fld, flname), os.path.join(dst_fld, flname))

    #delete files if they exceed size limit
    fls = os.listdir(dst_fld)
    file_n_size = [(f,os.stat(os.path.join(dst_fld, f)).st_size) for f in fls]
    #print(file_n_size)
    
    file_df = pd.DataFrame(file_n_size,columns=['file','size'])
    file_df = file_df.sort_values(by=['size'])
    file_df['total_size'] = file_df['size'].cumsum()
    file_df['to_delete'] = file_df['total_size'] < size_limit*10^6
    
    del_df = file_df[file_df['to_delete'] == True]
    if del_df.shape[0] > 0:
        for idx, row in del_df.iterrows():
            flname = row['file']
            os.remove(os.path.join(dst_fld, flname))
    
    print(file_df.tail(5))
    #return(file_df)


In [ ]:
# %%time

# sample_training_data(src_fld="../data/samples/trainset1", dst_fld="../data/train_sample")

# Segregate data function

#### This function segregates data into templates.

In [ ]:
# function to read cluster id and segregate the data

def segregate_data(src_dir, result_dir, cluster_dir, prefix, cluster_file):
    
    fls = glob.glob(os.path.join(result_dir,"*.json"))

    cols = ['filename','clusterId']
    clusters = pd.DataFrame(columns=cols)
    moved_file_cnt = 0

    for fl in fls:

        f = open(fl)
        dat = json.load(f)
        if dat['status'] == "succeeded":
            cid = dat['analyzeResult']['pageResults'][0]['clusterId']
            flname = os.path.basename(fl)
            tdf = pd.DataFrame([[flname, cid]], columns=cols)
            clusters = clusters.append(tdf, ignore_index=True)
            #print("clusterID for: "+fl+" is: "+str(cid), not (cid is None))

            if not (cid is None):
                #move files with cluster ID to cluster folders
                pdf_name = flname.replace(".json",".pdf")
                fld_name = prefix+"-"+str(cid)
                fld_path = os.path.join(cluster_dir,fld_name)
                if not os.path.isdir(fld_path):
                    os.makedirs(fld_path)
                    print("creating folder:"+fld_path)
                if os.path.isfile(os.path.join(src_dir,pdf_name)):
                    shutil.move(os.path.join(src_dir,pdf_name), os.path.join(fld_path,pdf_name))
                    #print(os.path.join(src_dir,pdf_name), os.path.join(fld_path,pdf_name))
                    moved_file_cnt = moved_file_cnt + 1

    clusters.to_csv(cluster_file)
    
    return(moved_file_cnt)
